In [1]:
from pyspark import SparkContext
from pyspark.sql.functions import countDistinct
sc=SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/28 04:41:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/28 04:41:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, bround
spark = SparkSession.builder \
         .appName('SparkByExamples.com') \
         .getOrCreate()
import pyspark.sql.functions as f
from pyspark import Row
from pyspark.shell import spark


22/12/28 04:41:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.1
      /_/

Using Python version 3.9.7 (default, Sep 16 2021 08:50:36)
Spark context Web UI available at http://192.168.178.88:4041
Spark context available as 'sc' (master = local[*], app id = local-1672198899311).
SparkSession available as 'spark'.


Pseude Code:

agrupar por user, device, gt
    para x, y, z sacar
        median
        desv
        max
        min

join data from different datasets into RDDs
Analize RDDs

Don´t repeat code!
avoid many (unnecessary) lines of code / repetitions

Define file names:

In [3]:
pa = 'Phones_accelerometer.csv'
pg = 'Phones_gyroscope.csv'
wa = 'Watch_accelerometer.csv'
wg = 'Watch_gyroscope.csv'

Define column names:

In [4]:
columns = ["index", "arrival_time", "creation_time", "x", "y", "z", "user", "model", "device", "activity"]


Define which columns should be float for preprocessing:

In [5]:
to_float = ["x", "y", "z"]

In [12]:
def preprocess_DF(file_name):
    """
    This function takes in the file names and puts out the pre-processed dateframe.
    """

    # read file in pyspark context
    read_file = sc.textFile(file_name)
    # split into columns and convert to DF
    DF = read_file.map(lambda line: [x for x in line.split(',')]).toDF(columns)
    
    #drop unnecessary columns
    DF_prep = DF.drop("index", "arrival_time", "creation_time", "device")
    #convert numerical columns to float
    for column in to_float:
        DF_prep = DF_prep.withColumn(column, col(column).cast('float'))

    return DF_prep

"key" defines, which columns make up the key for the analysis.

In [7]:
key = ["user", "model", "activity"]

"values" defines, which columns should contain numerical values.

In [8]:
values = ["avg(x)", "avg(y)", "avg(z)", "stddev(x)", "stddev(y)", "stddev(z)", "max(x)", "max(y)", "max(z)", "min(x)", "min(y)", "min(z)"]

In [13]:
def analize(input):
  """
  This function takes in the pre-processed data frame and performs the mathematical analysis:
  """

  #compute the mean, standard deviation, max, and min by aggregation on the key
  avg = input.groupBy(key).agg({"x":"avg", "y":"avg", "z":"avg"})
  std = input.groupBy(key).agg({"x":"stddev", "y":"stddev", "z":"stddev"})
  max = input.groupBy(key).agg({"x":"max", "y":"max", "z":"max"}) 
  min = input.groupBy(key).agg({"x":"min", "y":"min", "z":"min"})

  #join individual outcomes
  out = avg.join(std,key).join(max, key).join(min, key)

  #transform numerical values to double data type and round to 5 decimal places
  for col in values:
    out = out.withColumn(col, f.round(f.col(col).cast("double"), 5))


  return out

22/12/28 07:15:16 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 285134 ms exceeds timeout 120000 ms
22/12/28 07:15:16 WARN SparkContext: Killing executors is not supported by current scheduler.


In [10]:
def process(df):
    """
    Bundle the pre-processing and analytics functions.
    """
    pa_prep = preprocess_DF(df)
    out_pa = analize(pa_prep)

    return out_pa

Apply processing function to all data sets:

In [11]:
out_wg = process(wg)
out_pg = process(pg)
out_wa = process(wa)
out_pa = process(pa)

print(
    out_wg.show(),
    out_pg.show(),
    out_wa.show(),
    out_pa.show(),
)

+----+-----+--------+-------+--------+--------+---------+---------+---------+-------+--------+--------+--------+--------+--------+
|user|model|activity| avg(x)|  avg(y)|  avg(z)|stddev(x)|stddev(y)|stddev(z)| max(x)|  max(y)|  max(z)|  min(x)|  min(y)|  min(z)|
+----+-----+--------+-------+--------+--------+---------+---------+---------+-------+--------+--------+--------+--------+--------+
|   a| gear|   stand|0.02215|-0.03287|-0.07144|  0.07316|  0.05571|  0.05018| 0.8104| 0.35447| 1.14756|-2.03838|-0.55287|-1.23198|
|   a| gear|    null|0.02257|-0.03661|-0.07169|  0.01783|  0.01158|  0.00805|0.05619|-0.01811|-0.05513|-0.03143|-0.08043|-0.09348|
|   a| gear|     sit|0.02271|-0.04142|-0.06997|  0.05704|  0.05628|  0.04524|0.39868| 0.39921| 0.77605|-0.70094|-0.79043| -0.2285|
+----+-----+--------+-------+--------+--------+---------+---------+---------+-------+--------+--------+--------+--------+--------+

+----+------+--------+-------+-------+------+---------+---------+---------+-------